In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline
import seaborn as sns
import pandas as pd
import re
import json
import sqlite3
sns.set_style('darkgrid')

In [ ]:
import sys
sys.path.insert(1, '../../pytpc')

In [ ]:
import pytpc

# Output database connection

In [ ]:
gasdb = sqlite3.connect('../pytpc/data/gases/gasdata.db')
gdb_curs = gasdb.cursor()

# Read data sources

### Note on units

The units need to be kept consistent between ASTAR and SRIM sources.

Use units of MeV (total) for energy, $\mathrm{MeV/(g/cm^2)}$ for stopping power, and $\mathrm{cm/(g/cm^3)}$ for range.

In [ ]:
def read_astar(fp):
    return pd.read_table(fp, delim_whitespace=True, skiprows=5, names=('energy', 'dedx', 'range'), index_col=0)

In [ ]:
def read_srim(fp):
    en_units = {'keV': 1e-3, 'MeV': 1}
    dist_units = {'um': 1e-4, 'mm': 1e-1, 'cm': 1, 'm': 1e2}

    res = []

    with open(fp) as f:
        for line in f:
            if 'Density' in line:
                litems = line.strip().split()
                gas_dens = float(litems[3]) 
                print(gas_dens)
                assert litems[4] == 'g/cm3', 'Units for density are not g/cm3: {}'.format(litems[4])
            if 'Straggling' in line:
                f.readline()
                break
        for line in f:
            if '-------' in line:
                break
            litems = line.strip().split()
            if len(litems) != 10:
                raise ValueError('Wrong number of entries in line!')

            en = float(litems[0]) * en_units[litems[1]]
            dedx_elec = float(litems[2]) * 1000  # convert MeV/(mg/cm^2) to MeV/(g/cm^2)
            dedx_nuc = float(litems[3]) * 1000
            proj_range = float(litems[4]) * dist_units[litems[5]] * gas_dens

            res.append([en, dedx_elec + dedx_nuc, proj_range])

    res = pd.DataFrame(res, columns=('energy', 'dedx', 'range')).set_index('energy')
    
    return res

## Helium gas

This is for pure helium gas

### Alpha particles

In [ ]:
he_data = read_astar('../data/raw/helium_astar.txt')
he_data['proj_mass'] = 4
he_data['proj_charge'] = 2

In [ ]:
he_data.to_sql('helium', gasdb, if_exists='replace')

In [ ]:
gdb_curs.execute('INSERT INTO masses VALUES ("helium", 4.002)')
gasdb.commit()

In [ ]:
hegas = pytpc.gases.InterpolatedGas('helium', 760)
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.energy_loss(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')

In [ ]:
hegas = pytpc.gases.InterpolatedGas('helium', 760)
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.range(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('Range at 760 torr [m]')

## CO2 data

This is for pure CO2 gas

### Alpha particles

In [ ]:
co2_data = read_astar('../data/raw/co2_astar.txt')
co2_data['proj_mass'] = 4
co2_data['proj_charge'] = 2

In [ ]:
co2_data.to_sql('carbon_dioxide', gasdb, if_exists='replace')

In [ ]:
gdb_curs.execute('INSERT INTO Masses VALUES ("carbon_dioxide", 44.01)')
gasdb.commit()

In [ ]:
co2gas = pytpc.gases.InterpolatedGas('carbon_dioxide', 760)
ens = np.logspace(-3, 3, 100)
plt.plot(ens, co2gas.energy_loss(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')

In [ ]:
co2gas = pytpc.gases.InterpolatedGas('carbon_dioxide', 760)
ens = np.logspace(-3, 3, 100)
plt.plot(ens, co2gas.range(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('Range at 760 torr [m]')

## (Iso)butane gas

### Alpha particles

In [ ]:
but_alpha = read_srim('../pytpc/data/raw/srim-he4-butane.txt')
but_alpha['proj_mass'] = 4
but_alpha['proj_charge'] = 2

### Argon-40

In [ ]:
but_ar40 = read_srim('../pytpc/data/raw/srim-ar40-butane.txt')
but_ar40['proj_mass'] = 40
but_ar40['proj_charge'] = 18

# Argon-46

In [ ]:
but_ar46 = read_srim('../pytpc/data/raw/srim-ar46-butane.txt')
but_ar46['proj_mass'] = 46
but_ar46['proj_charge'] = 18

### Protons

In [ ]:
but_p = read_srim('../pytpc/data/raw/srim-p-butane.txt')
but_p['proj_mass'] = 1
but_p['proj_charge'] = 1

In [ ]:
butane = pd.concat((but_alpha, but_ar40, but_ar46, but_p), axis=0)
butane.to_sql('isobutane', gasdb, if_exists='replace')

In [ ]:
gdb_curs.execute('INSERT INTO Masses VALUES ("isobutane", 58.12)')
gasdb.commit()

In [ ]:
isogas = pytpc.gases.InterpolatedGas('isobutane', 18.)

In [ ]:
ens = np.logspace(-2, 2, 100)
plt.plot(ens, isogas.energy_loss(ens, 4, 2))
plt.plot(ens, isogas.energy_loss(ens, 40, 18))
plt.plot(ens, isogas.energy_loss(ens, 46, 18))
plt.plot(ens, isogas.energy_loss(ens, 1, 1))
plt.loglog()

# Gas Mixtures

## Helium-CO2 90/10

In [ ]:
heco2 = pytpc.gases.InterpolatedGasMixture(760, ('helium', 0.9), ('carbon_dioxide', 0.1))
ens = np.logspace(-3, 3, 100)
plt.plot(ens, heco2.energy_loss(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, heco2.range(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('Range at 760 torr [m]')

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.energy_loss(ens, 4, 2), label='Pure He')
plt.plot(ens, co2gas.energy_loss(ens, 4, 2), label='Pure CO2')
plt.plot(ens, heco2.energy_loss(ens, 4, 2), label='He-CO2 90/10')
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')
plt.legend()

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.energy_loss(ens, 4, 2)/co2gas.energy_loss(ens, 4, 2))
plt.loglog()

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.range(ens, 4, 2), label='Pure He')
plt.plot(ens, co2gas.range(ens, 4, 2), label='Pure CO2')
plt.plot(ens, heco2.range(ens, 4, 2), label='He-CO2 90/10')
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('Range at 760 torr [m]')
plt.legend(loc=2)
# plt.savefig('/Users/josh/Desktop/range_fullpress.pdf', transparent=True)

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, heco2.components[0][0].energy_loss(ens, 4, 2), 
         label='Pure Helium at {} torr'.format(heco2.components[0][1] * 760))
plt.plot(ens, heco2.components[1][0].energy_loss(ens, 4, 2), 
         label='Pure CO2 at {} torr'.format(heco2.components[1][1] * 760))
plt.plot(ens, heco2.energy_loss(ens, 4, 2), label='He-CO2 90/10 at 760 torr')
plt.loglog()
plt.xlabel('de/dx [MeV/m]')
plt.ylabel('Range [m]')
plt.legend(loc=1)

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, heco2.components[0][0].range(ens, 4, 2), 
         label='Pure Helium at {} torr'.format(heco2.components[0][1] * 760))
plt.plot(ens, heco2.components[1][0].range(ens, 4, 2), 
         label='Pure CO2 at {} torr'.format(heco2.components[1][1] * 760))
plt.plot(ens, heco2.range(ens, 4, 2), label='He-CO2 90/10 at 760 torr')
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('Range [m]')
plt.legend(loc=2)

In [ ]:
plt.plot(ens, heco2.components[0][0].energy_loss(ens, 4, 2) / heco2.components[1][0].energy_loss(ens, 4, 2))
plt.semilogx()

In [ ]:
%%timeit
ens = np.logspace(-3, 3, 500)
ranges = heco2.range(ens, 4, 2)
heco2_table = pd.DataFrame(np.vstack((ens, ranges)).T, columns=('energy', 'range'))

In [ ]:
InterpolatedUnivariateSpline(ranges, ens)

In [ ]:
rgs = np.linspace(0, 1, 100)
plt.plot(rgs, heco2.inverse_range(rgs, 4, 2))

In [ ]:
heco2_table.to_csv('/Users/josh/Desktop/heco2_range_table.csv', index=False)

# Comparing SRIM to data

In [ ]:
srim_heco_table = read_srim('/Volumes/bradt/Documents/Data/GasData/srim-he-he9co2.txt')
heco2 = pytpc.gases.InterpolatedGasMixture(299.5, ('helium', 0.9), ('carbon_dioxide', 0.1))

In [ ]:
srim = InterpolatedUnivariateSpline(srim_heco_table.index, srim_heco_table.dedx)

In [ ]:
ens = np.logspace(-3, 2, 100)
plt.plot(ens, heco2.energy_loss(ens, 4, 2) / heco2.density / 100, label='Weighted Combination')
plt.plot(ens, srim(ens), label='SRIM Compound')
plt.loglog()
plt.xlabel('Stopping Power [MeV/(g/cm^2)]')
plt.ylabel('Particle Kinetic Energy [MeV]')
plt.legend(loc=3, frameon=True)

plt.savefig('/Users/josh/Desktop/srim_plot.pdf')

In [ ]:
heco2.density